### Projection onto the symmetric subspace

In [1]:
import json
import numpy as np
from twirler.symmetry_groups import create_symmetric_group, create_induced_subgroup_from_permutations
from twirler.induced_representation import derive_unitaries_angle_embedding_analytic
from twirler.generators import get_ansatz_generators
from ansatz import Ansatz
import pennylane as qml

from twirler.twirling import apply_twirling_to_generators

n_qubits = 5
S = create_symmetric_group(n_qubits)

subgroups = None
with open(f"groups\\modified_subgroup_generators_{n_qubits}.json", "r") as f:
    subgroups = json.load(f)

In [2]:
subgroup_unitaries = {}
for k, groups in subgroups.items():
    subgroup_unitaries[k] = []
    for generators in groups:
        new_generators = []
        for g in generators:
            new_generators.append(tuple(g))
        K = create_induced_subgroup_from_permutations(S, new_generators)
        unitaries = derive_unitaries_angle_embedding_analytic(K)
        subgroup_unitaries[k].append({"unitaries": unitaries, "subgroup": K})

In [3]:
results = {}

depth = 1
for ansatz_id in range(1, 20):
    results[ansatz_id] = {}
    super_ansatz = Ansatz(ansatz_id, n_qubits, depth)
    ansatz = super_ansatz.get_ansatz()
    ansatz_generators = get_ansatz_generators(ansatz)

    commuted_generators = {}

    for k in subgroup_unitaries:
        total_norm = 0.0
        for elem in subgroup_unitaries[k]:
            unitaries = elem["unitaries"]
            subgroup = elem["subgroup"]
            twirled_generators = apply_twirling_to_generators(unitaries, ansatz_generators, len(subgroup.elements), n_qubits)

            # Compute D_orig = (1 / N_G) * sum_i ||G_i - G_i_symm||_F
            for gen_idx, (gen_observable, wires, gate_name, theta) in enumerate(ansatz_generators):
                op = qml.Hermitian(gen_observable, wires=wires)
                G_full = qml.matrix(op, wire_order=range(n_qubits))

                G_twirled = twirled_generators[gen_idx]['averaged']

                norm = np.linalg.norm(G_full - G_twirled, ord='fro')
                total_norm += norm

        avg_norm = total_norm / (len(ansatz_generators) * len(subgroup_unitaries[k]))

        results[ansatz_id][k] = avg_norm

In [4]:
for ansatz_id in results:
    print(f"Ansatz {ansatz_id}:")
    for s in results[ansatz_id]:
        print(f"  Subgroup size {s}: Average difference between generators = {results[ansatz_id][s]}")

Ansatz 1:
  Subgroup size 1: Average difference between generators = 0.0
  Subgroup size 2: Average difference between generators = 1.28
  Subgroup size 3: Average difference between generators = 1.385640646055102
  Subgroup size 4: Average difference between generators = 1.7797958971132657
  Subgroup size 5: Average difference between generators = 2.5298221281346995
  Subgroup size 6: Average difference between generators = 1.9189739793884295
  Subgroup size 8: Average difference between generators = 1.959591794226542
  Subgroup size 10: Average difference between generators = 2.5298221281346995
  Subgroup size 12: Average difference between generators = 2.1102910287789105
  Subgroup size 20: Average difference between generators = 2.5298221281346995
  Subgroup size 24: Average difference between generators = 1.959591794226541
  Subgroup size 60: Average difference between generators = 2.529822128134703
  Subgroup size 120: Average difference between generators = 2.529822128134703
Ans